In [ ]:
from PIL import Image
import numpy as np
import cv2

In [ ]:
def load_img(impath):
    """
    Loads an image from a specified location and returns it in RGB format.
    Input:
    - impath: a string specifying the target image location.
    Returns an RGB image.
    """
    image_bgr = cv2.imread(impath, cv2.IMREAD_COLOR)        # cv2 reads image in bgr format
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)  # convert to rgb format
    return image_rgb

In [ ]:
# Formerly used to mask reconstructed hands
def image_difference(image_path1, image_path2, output_path):
    """
    Takes the difference between two images and saves it as a JPG.
    Inputs:
    - image_path1: a string specifying the path to the first image file
    - image_path2: a string specifying the path to the second image file
    - output_path: a string specifying the path to where the difference of the two images will be saved
    """
    img1 = load_img(image_path1)
    img2 = load_img(image_path2)

    # Ensure both images are the same size
    if img1.size != img2.size:
        raise ValueError("Images must be the same size.")

    # Calculate the difference using numpy
    arr1 = np.array(img1, dtype=np.int16)
    arr2 = np.array(img2, dtype=np.int16)
    diff = (arr1 - arr2).astype(np.uint8)

    # Convert back to an image
    diff_image = Image.fromarray(diff, mode="RGB")

    # Save the output
    diff_image.save(output_path, "JPEG")

In [ ]:
def calculate_mse(image_path1, image_path2):
    """
    Calculates the mean square error,
    which is normalized by the number of pixels taken up by the reconstructed hands
    Inputs:
    - image_path1: a string specifying the path to the first image file
    - image_path2: a string specifying the path to the second image file
    """
    # Load images
    img1 = load_img(image_path1)
    img2 = load_img(image_path2)
    [h, w, _] = img2.shape
    
    # Ensure both images are the same size
    if img1.size != img2.size:
        raise ValueError("Images must be the same size.")

    # Count number of pixels in hand
    handpixels = 0
    for x in range(w):
        for y in range(h):
            if img1[y,x,0] > 25:    # Pixels that are not masked
                handpixels += 1

    # Calculate MSE
    mse = 0
    for x in range(w):
        for y in range(h):
            squared_diff = (img1 -img2) ** 2
            mse = np.sum(squared_diff)
            
    normalized_mse = mse /(handpixels)
    
    return normalized_mse